In [14]:
!singularity exec \
    --cleanenv \
    --containall \
    -H $PWD:/home \
    --nv \
    stress_slurm/gan.sif \
    python3.8 main.py --id 1 --saving

2023-11-13 15:39:17.781541: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
***Running scenario 1: 1-LOSO_15real
**Starting experiment run (1/12): CNN-LSTM_LOSO_15real...
*** Adding real data from: WESAD ***
*** Loaded 3 real and 0 synthetic subjects ***

*CI run 1/2...
2023-11-13 15:39:26.975511: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9803 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:06:00.0, compute capability: 7.5
2023-11-13 15:39:27.630068: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must fe

In [2]:
#!cd stress_slurm \
#    && singularity exec gan.sif spython recipe Dockerfile > Singularity \
#    && singularity build gan_.sif Singularity